In [6]:
%matplotlib inline
import pandas as pd
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

import warnings; warnings.simplefilter('ignore')

In [9]:
movie_metadata = pd.read_csv('movies.csv')
movie_links = pd.read_csv('links.csv')

In [10]:
movie_metadata.head(5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [14]:
movie_links.head(5)

0      862
1     8844
2    15602
3    31357
4    11862
Name: tmdbId, dtype: int64

In [11]:
movie_links = movie_links[movie_links['tmdbId'].notnull()]['tmdbId'].astype('int')
movie_metadata = movie_metadata.drop([19730, 29503, 35587])
movie_metadata['id'] = movie_metadata['id'].astype('int')
# pmm  (processed_movie_metadata)
pmm = movie_metadata[movie_metadata['id'].isin(movie_links)]

In [12]:
pmm['tagline'] = pmm['tagline'].fillna('')
pmm['overview'] = pmm['overview'].fillna('')
pmm['description'] = pmm['overview'] + pmm['tagline']
pmm['description'] = pmm['description'].fillna('')

In [25]:
vectorizer = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0.0, stop_words='english')
tfidf_matrix = vectorizer.fit_transform(pmm['description'])
print(tfidf_matrix.shape)

(45463, 1104516)


In [26]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [27]:
pmm = pmm.reset_index()
titles = pmm['title']
indices = pd.Series(pmm.index, index=pmm['title'])

In [33]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:20]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [42]:
answers = get_recommendations('Ip Man')

In [37]:
import joblib 
joblib.dump(cosine_sim, "./nlp_recommendation_cosine_sim.joblib", compress=True)

['./nlp_recommendation_cosine_sim.joblib']

In [41]:
joblib.dump(titles, "./titles.joblib", compress=True)
joblib.dump(indices, "./indices.joblib", compress=True)

['./indices.joblib']

In [39]:
titles

0                          Toy Story
1                            Jumanji
2                   Grumpier Old Men
3                  Waiting to Exhale
4        Father of the Bride Part II
                    ...             
45458                         Subdue
45459            Century of Birthing
45460                       Betrayal
45461               Satan Triumphant
45462                       Queerama
Name: title, Length: 45463, dtype: object

In [48]:
'Wing Chun' in answers.values

True